In [12]:
import os
import sys
import joblib
import FE_0823 as FE
import numpy as np
import pandas as pd
from tqdm import tqdm

In [15]:
import imp
imp.reload(FE)

<module 'FE_0823' from '/Users/jinseok/Jupyter/bigcon/Shopping_GG/2. FE/FE_0823.py'>

In [3]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [4]:
sale = pd.read_excel(os.path.join('..', '0.Data', '01_제공데이터', 'sale_data_v05_0828.xlsx'))
meta = pd.read_excel(os.path.join('..', '0.Data', '01_제공데이터', 'sale_meta_v04_0823.xlsx'))
item = pd.read_excel(os.path.join('..', '0.Data', '01_제공데이터', 'item_meta_v04_0828.xlsx'))

In [5]:
sale.shape, meta.shape, item.shape

((38309, 16), (2042, 15), (1364, 15))

# 01. Feature Engineering

In [6]:
# 재빈쓰
sale = FE.engineering_TimeDiff(sale)

In [7]:
sale = FE.engineering_Soldout(sale)

In [8]:
sale = FE.engineering_DatePrice(sale)

연도입력


 2019


In [17]:
sale = FE.engineering_trend(sale)

/Users/jinseok/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/polynomial.py:630: RuntimeWarning: invalid value encountered in true_divide
  lhs /= scale


In [21]:
sale

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,NEW상품코드,NEW상품명,...,NEW_평균판매단가,NEW_중간판매단가,NEW_최고-최저,NEW_분산,NEW_표준편차,상품군_가격대,전체_가격대,상품군평균판매단가차이,결합상품,log최근3개월상품군추세
0,2019-01-01 06:00:00,20.000,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.000,865.000,테이트 남성 셀린니트 3종,...,39900.000,39900.000,0.000,0.000,0.000,0.000,0.000,76662.983,0,0
1,2019-01-01 06:00:00,20.000,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.000,866.000,테이트 여성 셀린니트 3종,...,39900.000,39900.000,0.000,0.000,0.000,0.000,0.000,76662.983,0,0
2,2019-01-01 06:20:00,20.000,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.000,865.000,테이트 남성 셀린니트 3종,...,39900.000,39900.000,0.000,0.000,0.000,0.000,0.000,76662.983,0,0
3,2019-01-01 06:20:00,20.000,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.000,866.000,테이트 여성 셀린니트 3종,...,39900.000,39900.000,0.000,0.000,0.000,0.000,0.000,76662.983,0,0
4,2019-01-01 06:40:00,20.000,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.000,865.000,테이트 남성 셀린니트 3종,...,39900.000,39900.000,0.000,0.000,0.000,0.000,0.000,76662.983,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38304,2020-01-01 00:20:00,20.000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,nan,nan,삼성화재 행복한파트너 주택화재보험(1912),...,nan,nan,nan,nan,nan,nan,nan,nan,0,0
38305,2020-01-01 00:40:00,20.000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,nan,nan,삼성화재 행복한파트너 주택화재보험(1912),...,nan,nan,nan,nan,nan,nan,nan,nan,0,0
38306,2020-01-01 01:00:00,20.000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,nan,nan,삼성화재 행복한파트너 주택화재보험(1912),...,nan,nan,nan,nan,nan,nan,nan,nan,0,0
38307,2020-01-01 01:20:00,20.000,100490,201478,더케이 예다함 상조서비스(티포트),무형,0,nan,nan,더케이 예다함 상조서비스(티포트),...,nan,nan,nan,nan,nan,nan,nan,nan,0,0
